In [ ]:
from gasp.gt.nop.cls import k_means

k_means(
    '/mnt/d/gpto/caplivro_landscape/carta_clip.tif',
    '/mnt/d/gpto/caplivro_landscape/carta_kmeans.tif', 12
)

In [ ]:
"""
Random forest - Procude model file
"""

train_ref = '/home/jasp/mrgis/clsimg/lsb_train_f2/rst_train_f2.tif'

train_var = [
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b02_20180321.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b02_20180619.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b02_20181022.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b03_20180321.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b03_20180619.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b03_20181022.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b04_20180321.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b04_20180619.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b04_20181022.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b08_20180321.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b08_20180619.tif',
    '/home/jasp/mrgis/clsimg/lsb_imgcls/b08_20181022.tif',
]

outfile = '/home/jasp/mrgis/clsimg/mdl_trainfilter1.bz2'
outtbl = '/home/jasp/mrgis/clsimg/separability_lsb_osmdt_1.xlsx'

In [ ]:
from osgeo import gdal, gdal_array
import numpy as np
import pandas as pd
import os
import rpy2.robjects as robjects
from rpy2.robjects import numpy2ri
from rpy2.robjects.packages import importr
from gasp.to import obj_to_tbl

# Import package
sp = importr('spatialEco')

numpy2ri.activate()

separability = robjects.r['separability']

# Open data
img_ref = gdal.Open(train_ref, gdal.GA_ReadOnly)
img_var = [gdal.Open(i, gdal.GA_ReadOnly) for i in train_var]

# Data To Array
num_ref = img_ref.GetRasterBand(1).ReadAsArray()
num_ref = num_ref.reshape((-1, 1))
img_num = [x.GetRasterBand(1).ReadAsArray() for x in img_var]
img_num = [x.reshape((-1, 1)) for x in img_num]

# Get Classes codes
nd_val = img_ref.GetRasterBand(1).GetNoDataValue()

classes = np.unique(num_ref)
classes = classes[classes != nd_val]

# Create samples for each class
cls_samples = {}
for cls in classes:
    for v in range(len(img_num)):
        if not v:
            cls_samples[cls] = [img_num[v][num_ref == cls]]
        else:
            cls_samples[cls].append(img_num[v][num_ref == cls])

# Get separability matrix - one for each variable samples
mtxs = []
for v in range(len(img_num)):
    mtx = []
    for i in range(classes.shape[0]):
        row = []
        for e in range(classes.shape[0]):
            if i < e:
                sep_val = None
            else:
                b, jm, m, mdif, d, td = separability(
                    cls_samples[classes[i]][v],
                    cls_samples[classes[e]][v]
                )[0]
            
                sep_val = td
        
            row.append(sep_val)
        mtx.append(row)
    mtxs.append(pd.DataFrame(mtx, index=classes, columns=classes))

for df in range(len(mtxs)):
    mtxs[df]['classe'] = mtxs[df].index

# Export result
obj_to_tbl(mtxs, outtbl, sheetsName=[os.path.basename(f)for f in train_var])

In [ ]:
mtx.head(10)

In [ ]:
import rpy2.robjects as robjects
from rpy2.robjects import numpy2ri
from rpy2.robjects.packages import importr
import numpy as np

numpy2ri.activate()

# Create vector's
v1 = np.array([1362,1411,1457,1735,1621,1621,1791,1863,1863,1838])
v2 = np.array([1362,1411,1457,10030,1621,1621,1791,1863,1863,1838, 1837, 1836])
#v1 = robjects.IntVector([1362,1411,1457,1735,1621,1621,1791,1863,1863,1838])
#v2 = robjects.IntVector([1362,1411,1457,10030,1621,1621,1791,1863,1863,1838])
#v1 = robjects.IntVector([0,3,4,5,6,7,8,9,90,65])
#v2 = robjects.IntVector([1362,1411,13445,1735,1621,1621,1791,1863,1863,1838])



# Calling function

In [ ]:
print(b)
print(jm)
print(m)
print(mdif)
print(d)
print(td)

In [ ]:
from osgeo import gdal, gdal_array
import numpy as np
import joblib
from sklearn.ensemble import RandomForestClassifier
import datetime

img_ref = gdal.Open(train_ref, gdal.GA_ReadOnly)
img_var = [gdal.Open(i, gdal.GA_ReadOnly) for i in train_var]

nd_val = img_ref.GetRasterBand(1).GetNoDataValue()

nvar = len(img_var)

# Convert imgs to Array, remove nodata values and reshape
num_ref = img_ref.GetRasterBand(1).ReadAsArray()
num_ref = num_ref.reshape((-1, 1))
Y = num_ref[num_ref != nd_val]

X = np.zeros(
    (Y.shape[0], nvar),
    gdal_array.GDALTypeCodeToNumericTypeCode(
        img_var[0].GetRasterBand(1).DataType
    )
)
for b in range(X.shape[1]):
    a = img_var[b].GetRasterBand(1).ReadAsArray()
    a = a.reshape((-1, 1))
    a = a[num_ref != nd_val]
    
    X[:, b] = a

In [ ]:
# Fit Model
time_a = datetime.datetime.now().replace(microsecond=0)

clf = RandomForestClassifier(
    n_estimators=1000, random_state=0, n_jobs=-1
)
clf.fit(X, Y)

time_b = datetime.datetime.now().replace(microsecond=0)

print(time_b - time_a)

In [ ]:
# Save model file
joblib.dump(clf, outfile)

In [ ]:
print(X.shape)
print(Y.shape)

In [ ]:
print(Y.shape)

In [ ]:
print(num_y.shape)